In [1]:
from utils import *

In [2]:
from azureml.core import Workspace

# Configure experiment
ws = Workspace.from_config()

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [3]:
# Create or get training cluster
aml_cluster = get_aml_cluster(ws, cluster_name="cpu-cluster")
aml_cluster.wait_for_completion(show_output=True)

InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
# Create a run configuration
run_conf = get_run_config(['numpy', 'pandas', 'scikit-learn', 'tensorflow'])

In [5]:
from azureml.core import Dataset

dataset = Dataset.get_by_name(ws, name='titanic')
data_in = dataset.as_named_input('titanic')

In [6]:
from azureml.core import Datastore
from azureml.pipeline.core import PipelineData

datastore = Datastore.get(ws, datastore_name="mldata")
data_train = PipelineData('train', datastore=datastore)
data_test = PipelineData('test', datastore=datastore)

In [22]:
from azureml.pipeline.steps import PythonScriptStep

step1 = PythonScriptStep(name='Preprocessing',
                          script_name="preprocess_output.py",
                          source_directory="code",
                          arguments=[
                              "--input", data_in,
                              "--out-train", data_train,
                              "--out-test", data_test],
                          inputs=[data_in],
                          outputs=[data_train, data_test],
                          runconfig=run_conf,
                          compute_target=aml_cluster)


In [34]:
from azureml.pipeline.steps import PythonScriptStep

def get_step(name):
    return PythonScriptStep(name=name,
                          script_name="train.py",
                          source_directory="code",
                          arguments=[
                              "--in-train", data_train,
                              "--in-test", data_test],
                          inputs=[data_train, data_test],
                          runconfig=run_conf,
                          compute_target=aml_cluster)

step2 = get_step("Training 1")
step3 = get_step("Training 2")
step4 = get_step("Training 3")

In [24]:
from azureml.pipeline.core import Pipeline

pipeline = Pipeline(ws, steps=[step1, step2, step3, step4])

In [25]:
pipeline.validate()

Step Training 1 is ready to be created [1d5b6494]Step Training 2 is ready to be created [9d906576]

Step Training 3 is ready to be created [c36b250f]


[]

In [26]:
from azureml.core import Experiment

exp = Experiment(ws, "azureml-pipeline")
run = exp.submit(pipeline)

Created step Preprocessing [a36c2006][debaed5b-722b-4a15-b19f-3b06d1615d90], (This step is eligible to reuse a previous run's output)Created step Training 1 [1d5b6494][ff3bcf86-deda-4bb1-b192-652d96843e24], (This step will run and generate new outputs)
Created step Training 2 [9d906576][1546ff38-cc27-4432-ac64-f125601bdb5c], (This step will run and generate new outputs)Created step Training 3 [c36b250f][e5b7e199-81aa-422a-978e-059de395f1e9], (This step will run and generate new outputs)


Submitted PipelineRun 16f22bb3-f292-46af-8609-aa4a8d3df796
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/16f22bb3-f292-46af-8609-aa4a8d3df796?wsid=/subscriptions/21dc412b-d9eb-42e7-8317-55bc8eb10cf5/resourcegroups/packt/workspaces/mldemows&tid=e702ab66-1ac1-42ba-b41b-8195d943a1e0


In [27]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [35]:
step3.run_after(step2)
step4.run_after(step3)

In [36]:
from azureml.pipeline.core import Pipeline

pipeline = Pipeline(ws, steps=[step1, step2, step3, step4])

In [37]:
pipeline.validate()

Step Training 2 is ready to be created [9a44a696]
Step Training 3 is ready to be created [1df3f7c4]


[]

In [38]:
from azureml.core import Experiment

exp = Experiment(ws, "azureml-pipeline")
run = exp.submit(pipeline)

Created step Preprocessing [95597b3a][debaed5b-722b-4a15-b19f-3b06d1615d90], (This step is eligible to reuse a previous run's output)Created step Training 1 [6316b8b4][ff3bcf86-deda-4bb1-b192-652d96843e24], (This step is eligible to reuse a previous run's output)
Created step Training 2 [9a44a696][079b6168-654b-43af-9b9c-3f023b42ea47], (This step will run and generate new outputs)
Created step Training 3 [1df3f7c4][6eae93c9-2854-4757-85d5-ec6588c70112], (This step will run and generate new outputs)

Submitted PipelineRun 946daf66-24a6-4369-8c7e-356bd0ec256d
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/946daf66-24a6-4369-8c7e-356bd0ec256d?wsid=/subscriptions/21dc412b-d9eb-42e7-8317-55bc8eb10cf5/resourcegroups/packt/workspaces/mldemows&tid=e702ab66-1ac1-42ba-b41b-8195d943a1e0


In [39]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …